## Задание: Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.


---
## Этапы исследования и выводы

1. **Подготовка данных**  
   - Данные разделены на тренировочные (75%) и валидационные (25%) выборки.  
   - Создан пайплайн для предобработки и нормализации данных.  
   - Исключены записи с повторяющимися идентификаторами.

2. **Базовая модель (DummyRegression)**  
   - Базовые значения RMSE: от 44.25 до 45.96 тысяч баррелей.  
   - Дамми модель позволила сравнить результаты разрабатываемые модели с простой стратегией.

3. **Обучение моделей**  
   - Использованы алгоритмы LinearRegression, Lasso и Ridge с гиперпараметрами для (`alpha`) и различных способов нормализации (StandardScaler, MinMaxScaler, RobustScaler).  
   - На каждом из трёх регионов выбрана лучшая модель, произведена оценка RMSE на тренировочной и валидационной выборках.  
   - Средние запасы нефти на одну скважину в регионах отличаются: ~69 тыс. баррелей (регион 2) против ~93 и ~95 тыс. баррелей (регионы 1 и 3).

4. **Результаты моделей**  
   - Лучше всего модель работает на втором регионе (RMSE ~1 тыс. баррелей), что составляет ~1.3% от среднего уровня запасов.  
   - В первом и третьем регионах RMSE в относительных величинах выше ~41% и ~43% от среднего запаса.  

5. **Подготовка к расчёту прибыли**  
   - Учтён бюджет в 10 млрд руб. и стоимость одной тысячи баррелей 450 тыс. руб.  
   - Рассчитан достаточный объём запасов скважины (111 тыс. баррелей) для безубыточной разработки 200 скважин.  
   - Средний требуемый уровень запасов выше среднего уровня запасов во всех регионах (93, 69 и 95 тыс. баррелей).

6. **Расчёт прибыли и рисков (Bootstrap)**  
   - Для каждого региона методом Bootstrap сформировано 1000 подвыборок по 500 скважин.  
   - Расчёт прибыли велся для 200 скважин с наибольшими прогнозными запасами.  
   - Средняя прибыль, доверительные интервалы и вероятность убытка исследованы для каждого региона.  

7. **Основной результат**  
   - К разработке рекомендован регион 2 как обладающий наивысшей потенциальной доходностью (441 млн. руб.) и наименьшим риском понести убыток (2%).
   
**Результаты расчтетов средней прибыли и риска понести убытки**

| Показатель                                                       | Регион 1         | Регион 2         | Регион 3         |
|------------------------------------------------------------------|------------------|------------------|------------------|
| **Средняя прибыль, руб.**                                        | 396_664_247.00   | 441_056_563.00   | 362_988_258.00   |
| **Нижняя граница дов. интервала (2.5%) прибыли, руб.**           | -123_203_852.00  | 44_287_187.00    | -137_892_472.00  |
| **Верхняя граница дов. интервала (97.5%) прибыли, руб.**         | 386_586_673.00   | 429_237_289.00   | 354_210_165.00   |
| **Риск убытка, %**                                               | 7.50             | 2.00             | 8.70             |

## Загрузка и подготовка данных

### Импорт библиотек

In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import (
    RobustScaler,
    StandardScaler,
    MinMaxScaler
)
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.model_selection import RandomizedSearchCV

from sklearn.dummy import DummyRegressor
from sklearn.linear_model import (
    LinearRegression,
    Lasso,
    Ridge
)

from sklearn.metrics import (
    root_mean_squared_error,
    make_scorer
)

seed = 42
state = np.random.RandomState(12345)
pd.options.display.float_format = '{:_.2f}'.format

### Загрузка данных

In [2]:
# тренировочная выборка
df_region_1 = pd.read_csv('https://code.s3.yandex.net/datasets/geo_data_0.csv')

# тестовая выборка (без  целевого признака)
df_region_2 = pd.read_csv('https://code.s3.yandex.net/datasets/geo_data_1.csv')

# целевой признак тестовой выборки
df_region_3 = pd.read_csv('https://code.s3.yandex.net/datasets/geo_data_2.csv')

dfs = {
    'df_region_1': df_region_1,
    'df_region_2': df_region_2,
    'df_region_3': df_region_3
}

In [3]:
for key, val in dfs.items():
    print(key)
    display(val)
    print('', '', sep='\n')

df_region_1


,id,f0,f1,f2,product
0,txEyH,0.71,-0.50,1.22,105.28
1,2acmU,1.33,-0.34,4.37,73.04
2,409Wp,1.02,0.15,1.42,85.27
3,iJLyR,-0.03,0.14,2.98,168.62
4,Xdl7t,1.99,0.16,4.75,154.04
...,...,...,...,...,...
99995,DLsed,0.97,0.37,6.08,110.74
99996,QKivN,1.39,-0.38,1.27,122.35
99997,3rnvd,1.03,0.02,-1.35,64.38
99998,7kl59,1.00,-0.53,1.58,74.04




df_region_2


,id,f0,f1,f2,product
0,kBEdx,-15.00,-8.28,-0.01,3.18
1,62mP7,14.27,-3.48,1.00,26.95
2,vyE1P,6.26,-5.95,5.00,134.77
3,KcrkZ,-13.08,-11.51,5.00,137.95
4,AHL4O,12.70,-8.15,5.00,134.77
...,...,...,...,...,...
99995,QywKC,9.54,-6.88,2.00,53.91
99996,ptvty,-10.16,-12.56,5.01,137.95
99997,09gWa,-7.38,-3.08,5.00,137.95
99998,rqwUm,0.67,-6.15,1.00,30.13




df_region_3


,id,f0,f1,f2,product
0,fwXo0,-1.15,0.96,-0.83,27.76
1,WJtFt,0.26,0.27,-2.53,56.07
2,ovLUW,0.19,0.29,-5.59,62.87
3,q6cA6,2.24,-0.55,0.93,114.57
4,WPMUX,-0.52,1.72,5.90,149.60
...,...,...,...,...,...
99995,4GxBu,-1.78,1.13,6.26,172.33
99996,YKFjq,-1.26,-0.89,2.52,138.75
99997,tKPY3,-1.20,-2.96,5.22,157.08
99998,nmxp2,-2.42,2.42,-5.55,51.80


**Загрузка данных**

Данные загружены корретктно и соответсвтуют описанию задачи.

### Предобработка данных

In [4]:
def df_summary(name: str, df: pd.DataFrame) -> None:
    """
    Функция df_summary принимает на вход str DataFrame и выводит три сводные таблицы с общей информацией:
    
    1. Первая таблица содержит:
       - Первые пять строк датафрейма
    
    2. Вторая таблица содержит:
       - Общее количество строк
       - Общее количество столбцов
       - Количество полных дубликатов строк
    
    я. Третяя таблица содержит информацию о каждом признаке:
       - Название признака
       - Количество пропусков для каждого признака
       - Количество уникальных значений для каждого признака
       - Тип данных каждого признака

    Параметры:
    name (str): Входная строка - название датафрейма;
    df (pd.DataFrame): Входной датафрейм, для которого нужно сформировать сводные таблицы.

    Возвращаемое значение:
    None. Функция выводит таблицы напрямую.
    """ 
    
    # Таблица 1: общее число строк, столбцов и дубликатов
    total_rows = df.shape[0]
    total_columns = df.shape[1]
    total_duplicates = df.duplicated().sum()
    
    summary_1 = pd.DataFrame({
        'Кол-во строк': [total_rows],
        'Кол-во признаков': [total_columns],
        'Кол-во полных дубликатов': [total_duplicates]
    })
    summary_1_t = summary_1.T
    summary_1_t.columns = ['Значение']

    # Таблица 2: информация о каждом признаке
    summary_2 = pd.DataFrame({
        'Признак': df.columns,
        'Кол-во пропусков': df.isnull().sum(),
        'Кол-во уникальных': df.nunique(),
        'Тип': df.dtypes
    }).reset_index(drop=True)
    

    # Вывод таблиц
    print(f'\n\nТаблица: {name}')
    
    print('Обзор данных')
    display(df)
    
    print('', 'Общая информация', sep='\n')
    display(summary_1_t)
    
    print('', 'Информация о признаках',sep='\n')
    display(summary_2)
    
    print('', '', sep='\n')
    
    # Вывод уникальных значений признаков
    print('Уникальные значения признаков')
    for col in df.columns:
        print('', col, df[col].sort_values().unique(), sep='\n')
    

In [5]:
for key, val in dfs.items():
    df_summary(key, val)



Таблица: df_region_1
Обзор данных


,id,f0,f1,f2,product
0,txEyH,0.71,-0.50,1.22,105.28
1,2acmU,1.33,-0.34,4.37,73.04
2,409Wp,1.02,0.15,1.42,85.27
3,iJLyR,-0.03,0.14,2.98,168.62
4,Xdl7t,1.99,0.16,4.75,154.04
...,...,...,...,...,...
99995,DLsed,0.97,0.37,6.08,110.74
99996,QKivN,1.39,-0.38,1.27,122.35
99997,3rnvd,1.03,0.02,-1.35,64.38
99998,7kl59,1.00,-0.53,1.58,74.04



Общая информация


,Значение
Кол-во строк,100000
Кол-во признаков,5
Кол-во полных дубликатов,0



Информация о признаках


...


Общая информация


,Значение
Кол-во строк,100000
Кол-во признаков,5
Кол-во полных дубликатов,0



Информация о признаках


,Признак,Кол-во пропусков,Кол-во уникальных,Тип
0,id,0,99996,object
1,f0,0,100000,float64
2,f1,0,100000,float64
3,f2,0,100000,float64
4,product,0,100000,float64




Уникальные значения признаков

id
['009Gl' '00AuD' '00CaL' ... 'zzqqy' 'zzsKd' 'zzz9h']

f0
[-8.76000362 -7.45058711 -7.18949804 ...  7.19461485  7.21552717
  7.23826248]

f1
[-7.08401976 -6.74835677 -6.73299712 ...  7.10161842  7.76185714
  7.84480127]

f2
[-11.97033454 -11.61169048 -11.40724351 ...  16.31301122  16.35764509
  16.73940206]

product
[0.00000000e+00 4.60600004e-03 9.20411196e-03 ... 1.90011722e+02
 1.90013589e+02 1.90029838e+02]


**Обзор данных**

1. Все три датасета состоят из 100 тыс. записей, 4-х описательных признаков и целевого признака.
2. Во всех 3 датасетах, в незначительном количестве (от 4-х до 10-и) присутствуют записи с одинаковыми идентификационными номерами, но не являющиеся полными дубликатами, что требует уточнения. До получения уточняющей информации исключим эти записи из дальнейшего анализа.

In [6]:
for df_name, df in dfs.items():
    df.drop_duplicates(subset='id', inplace=True)

In [7]:
for df_name, df in dfs.items():
    print(df_name, ':', df.shape)

df_region_1 : (99990, 5)
df_region_2 : (99996, 5)
df_region_3 : (99996, 5)


### Промежуточные выводы

1. Данные загружены корретктно и соответсвтуют описанию задачи.
2. Устранены записи с повторяющимися уникальными идентификационными номерами до получения уточняющей информации.

## Обучение и проверка модели

### Подготовка данных и пайплана

**Разделение данных на обучающую и валидационную выборки**

In [8]:
dfs_ready_to_go = {}
count = 1

for df_name, df in dfs.items():
    X = df.drop(['id', 'product'], axis=1)
    y = df['product']
    
    # Разбиваем данные
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.25, random_state=seed
    )
    
    # Сохраняем результат в словарь
    dfs_ready_to_go[f'region_{count}'] = [X_train, X_val, y_train, y_val]
    
    count += 1

In [9]:
for region, dfs in dfs_ready_to_go.items():
    for i in range(0, len(dfs)):
        print(region, dfs[i].shape)

region_1 (74992, 3)
region_1 (24998, 3)
region_1 (74992,)
region_1 (24998,)
region_2 (74997, 3)
region_2 (24999, 3)
region_2 (74997,)
region_2 (24999,)
region_3 (74997, 3)
region_3 (24999, 3)
region_3 (74997,)
region_3 (24999,)


**Пайплайн подготовки данных**

In [10]:
num_cols = ['f0', 'f1', 'f2']

In [11]:
num_pipe = Pipeline([
        ('simpleimputer_before',
         SimpleImputer(
             missing_values=np.nan, 
             strategy='most_frequent')
        ),
        ('encoder', None
        ),
        ('simpleimputer_after',
         SimpleImputer(
             missing_values=np.nan, 
             strategy='most_frequent')
        ),
])

In [12]:
data_preprocessor = ColumnTransformer([
        ('num', num_pipe, num_cols)], 
        remainder = 'passthrough'
)

In [13]:
pipe_final = Pipeline([
        ('preprocessor', data_preprocessor),
        ('models', None)
])

**Подготовка данных и пайплайна**

1. Данные разделены на тренировочные (0.75) и валидационные (0.25) выброрки
2. Создан пайплайн подготовки данных

### Метрика


In [14]:
rmse_scorer = make_scorer(root_mean_squared_error, greater_is_better=False)

**Метрика**

Метрика RMSE определена заказчиком.

### Дамми модель (DummyRegression)

In [15]:
count = 1

for region, dfs in dfs_ready_to_go.items():
    X_train = dfs[0]
    X_val = dfs[1]
    y_train = dfs[2]
    y_val = dfs[3]
    
    dummy_model = DummyRegressor(
        strategy='mean'
    )
    dummy_model.fit(X_train, y_train)

    # предсказание на тестовых данных
    dummy_model_preds = dummy_model.predict(X_val)

    # оценка качества модели по метрике rmse
    rmse_score = root_mean_squared_error(y_val, dummy_model_preds)

    print(f'Регион {count}')
    print('RMSE на валидационной выборке =', round(rmse_score, 2))
    
    count += 1

Регион 1
RMSE на валидационной выборке = 44.25
Регион 2
RMSE на валидационной выборке = 45.96
Регион 3
RMSE на валидационной выборке = 44.67


**Дамми модель**

1. DummyRegression обучена с целью предоставления базового уровня качества модели
2. Используемая стратегия: 'mean'
3. Модель демонстрирует значение RMSE:
    - регион 1 = 44.25
    - регион 2 = 45.96
    - регион 3 = 44.67

### Обучение модели и оценка результатов

**Сетка гиперпараметров**

In [16]:
# гиперпараметры для RandomizedSearchCV
params_grid = [
    {
        'preprocessor': [StandardScaler(), MinMaxScaler(), RobustScaler(), 'passthrough'],  
        'models': [LinearRegression()]
    },
    {
        'preprocessor': [StandardScaler(), MinMaxScaler(), RobustScaler(), 'passthrough'],
        'models': [Lasso(random_state=seed)],
        'models__alpha': np.arange(0.1, 1.0, 0.1)
    },
    {
        'preprocessor': [StandardScaler(), MinMaxScaler(), RobustScaler(), 'passthrough'],  
        'models': [Ridge(random_state=seed)],
        'models__alpha': np.arange(0.1, 1.0, 0.1)
    }
] 

**Обучение моделей и прогнозирование**

In [17]:
rs = RandomizedSearchCV(
    pipe_final, 
    params_grid, 
    cv = 5,
    scoring=rmse_scorer,
    n_jobs=-1
)

In [18]:
rss = {
    'region_1_rs': None,
    'region_2_rs': None,
    'region_3_rs': None
}

results = {}

count = 1


for region, dfs in dfs_ready_to_go.items():
    X_train = dfs[0]
    X_val = dfs[1]
    y_train = dfs[2]
    y_val = dfs[3]
    
    # Обучим и предскажем
    rs.fit(X_train, y_train)
    y_pred = rs.predict(X_val)
    
    # Добавляем столбцы с фактами и предсказаниями только после предсказаний
    X_val['fact'] = y_val   
    X_val['prediction'] = y_pred   

    # Посчитаем метрики
    rmse_train = round(rs.best_score_, 1) * -1
    rmse_val = round(root_mean_squared_error(y_val, y_pred), 1)
    mean_oil_reserve = round(y_pred.mean(), 1)
    mean_error_percent = round(rmse_val / mean_oil_reserve * 100, 1)
    
    # Сохранение результатов для текущего региона
    results[f'Регион {count}'] = {
        'Средний объем запасов нефти в скважине, тыс. баррелей': mean_oil_reserve,
        'RMSE (train)': rmse_train,
        'RMSE (val)': rmse_val,
        'Средняя ошибка модели (val), %': mean_error_percent
    }
    
    
    
    # Сохраним rs объекты в словарь
    if count == 1:
        rss['region_1_rs'] = rs
    elif count == 2:
        rss['region_2_rs'] = rs
    else:
        rss['region_3_rs'] = rs
    
    count += 1
    
results_df = pd.DataFrame(results)
results_df

,Регион 1,Регион 2,Регион 3
"Средний объем запасов нефти в скважине, тыс. баррелей",92.60,68.60,94.90
RMSE (train),37.70,0.90,40.00
RMSE (val),37.70,0.90,40.10
"Средняя ошибка модели (val), %",40.70,1.30,42.30


**Обучение модели и оценка результатов**

1. Модель линейной регрессии определена заказчиком
2. Для моделей LinearRegression, Lasso и Ridge определена сетка гиперпараметров:
    - этап подготовки данных (нормализация): StandardScaler, MinMaxScaler и RobustScaler
    - гиперпарамтры моделей Lasso и Ridge: `alpha` в диапазоне от 0.1 до 1 с шагом 0.1
3. Лучшая модель и гиперпараметры подобраны с помощью RandomizedSearchCV
4. В результате, для каждого региона:

    - определена и обучена лучшая модель
    - спрогнозированы объемы запасов нефти в каждой скважине
    - определены значения метрики RMSE на тренировочной и влидационной выборках
    - определен средни1 запас нефти в скважинах
5. В среднем в регионе 2 приходится меньше всего запасов нефти на свкажину - 69 тыс. баррелей против 93 и 95 тыс. баррелей в регионах 1 и 3 соотвесттвенно. Лучше всего модель справляется с прогнозом во втором регионе - значение RMSE составляет 1 тысяча баррелей или ~1,3% от среднего значения запаса в скважинах против ~41% и ~43% в регионах 1 и 3 соответственно.

### Промежуточные выводы

**1. Подготовка данных и пайплайна**
- Данные разделены на тренировочные (75%) и валидационные (25%) выборки.
- Создан пайплайн для подготовки данных.

**2. Метрика**
- Метрика RMSE выбрана в соответствии с требованиями заказчика.

**3. Дамми модель**
- Для базовой оценки качества моделей обучена DummyRegression с использованием стратегии `mean`.
- Результаты RMSE для DummyRegression:
  - Регион 1: 44.25
  - Регион 2: 45.96
  - Регион 3: 44.67

**4. Обучение модели и оценка результатов**
- Заказчиком определено использование модели линейной регрессии.
- Для моделей LinearRegression, Lasso и Ridge подобраны гиперпараметры с использованием RandomizedSearchCV:
  - Этап нормализации данных: StandardScaler, MinMaxScaler, RobustScaler.
  - Гиперпараметры для Lasso и Ridge: значение `alpha` в диапазоне от 0.1 до 1 с шагом 0.1.
- Итоги обучения моделей:
  - Для каждого региона определена лучшая модель и соответствующие гиперпараметры.
  - Выполнено прогнозирование объемов запасов нефти в каждой скважине.
  - Вычислены значения метрики RMSE на тренировочной и валидационной выборках.
  - Определены средние запасы нефти на скважину.
- Основные результаты:
  - Регион 2 обладает наименьшим средним запасом нефти на скважину — 69 тыс. баррелей против 93 и 95 тыс. баррелей в регионах 1 и 3 соответственно.
  - Модель наиболее успешно прогнозирует запасы нефти в регионе 2 — RMSE составляет всего 1 тыс. баррелей (~1.3% от среднего запаса в скважинах), в то время как в регионах 1 и 3 RMSE составляет около 41% и 43% от среднего значения запаса соответственно.
  - Прогнозные значения моделей лучше прогнозов дамми моделей. Применение моделей обосновано.

## Подготовка к расчёту прибыли

**Входные данные**

In [19]:
# кол-во исследуемых скважин
NUM_OF_WELLS_FOR_RESEARCH = 500

# кол-во скважин к разработке
NUM_OF_WELLS_FOR_DEVELOPMENT = 200

# бюджет на разработку скважин
BUDGET_RUB = 10_000_000_000

# цена реализации 1000 баррелей
INCOME_PER_UNIT_RUB = 450_000

# доверителтельный интервал
CONFIDENCE_INTERVAL = 0.95

**Порог безубыточности скважины, тыс. баррелей**

In [20]:
break_even_threshold_rub = BUDGET_RUB / NUM_OF_WELLS_FOR_DEVELOPMENT / INCOME_PER_UNIT_RUB

print(
    'Минимальное количество баррелей в скважине для безубыточной разработки 200 скважин',
    round(break_even_threshold_rub),
    'тыс. баррелей'
)

Минимальное количество баррелей в скважине для безубыточной разработки 200 скважин 111 тыс. баррелей


### Промежуточные выводы

1. Все ключевые значения для расчётов сохранены в отдельных переменных
    - количество исследуемых скважин - 500 шт.
    - количество скважин к разработке - 200 шт.
    - бюджет - 10 млрд. руб.
    - выручка за 1 тысю баррелей - 450 тыс. руб.
    - значение доверительного интервала - 95%
2. Рассчитан достаточный объём сырья для безубыточной разработки новой скважины - 111 тыс. баррелей, что выше средних значений полученных в результате прогноза: ~93, ~69 и ~95 тыс. баррелей в регионах 1, 2 и 3 соответственно

## Расчёт прибыли и рисков 

**Функция для расчет прибыли**

In [21]:
def profit(prediction, count):
    wells_sorted = prediction[['fact', 'prediction']].sort_values(by='prediction', ascending=False)
    selected = wells_sorted['fact'][:count]
    return round(INCOME_PER_UNIT_RUB * selected.sum() - BUDGET_RUB)

**Расчет максимально возможной прибыли**

In [22]:
count = 1

for region, dfs in dfs_ready_to_go.items():
    total_profit = profit(dfs[1][['fact', 'prediction']], 200)
    
    print(
        f'Максимально возможная прибыль в регионе {count} по топ 200 скважинам составляет',
        round(total_profit / 1_000_000_000, 1),
        'млрд. рублей'
    )
    
    count += 1

Максимально возможная прибыль в регионе 1 по топ 200 скважинам составляет 3.5 млрд. рублей
Максимально возможная прибыль в регионе 2 по топ 200 скважинам составляет 2.4 млрд. рублей
Максимально возможная прибыль в регионе 3 по топ 200 скважинам составляет 2.4 млрд. рублей


### Промежуточные выводы

Для каждого региона определена максимально возможная прибыль от разработки 200 скважин с максимальными запасами нефти определнные по результатам прогноза:
   - регион 1 - 3.5 млрд. руб.
   - регион 2 - 2,4 млрд. руб.
   - регион 3 - 2,4 млрд. руб.

## Риски и прибыль

**Статистический анализ средней прибыли и риска получить убыток для каждого региона методом bootstrap**

In [23]:
count = 1
results = {}

for region, dfs in dfs_ready_to_go.items():
    profits = []
    
    # Генерация прибылей с бутстрэпом
    for i in range(1000):
        subsample = dfs[1][['fact', 'prediction']].sample(
            n=500,
            replace=False,
            random_state=state
        )
        profits.append(profit(subsample, 200))
    
    # Преобразование прибылей в Series
    profits = pd.Series(profits)
    mean = round(profits.mean())
    lower = round(profits.quantile((1-CONFIDENCE_INTERVAL) / 2))
    upper = round(profits.quantile(1-(1-CONFIDENCE_INTERVAL / 2)))
    
    # Определение отрицательных элементов
    negative_ratio = (profits < 0).mean() * 100
    
    # Сохранение результатов для текущего региона
    results[f'Регион {count}'] = {
        'Средняя прибыль, руб.': mean,
        'Нижняя граница дов. интервала (2.5%) прибыли, руб.': lower,
        'Верхняя граница дов. интервала (97.5%) прибыли, руб.': upper,
        'Риск убытка, %': negative_ratio
    }
    
    count += 1

# Преобразование в DataFrame, где регионы — столбцы
results_df = pd.DataFrame(results)
results_df

,Регион 1,Регион 2,Регион 3
"Средняя прибыль, руб.",402_752_815.00,440_177_804.00,368_939_058.00
"Нижняя граница дов. интервала (2.5%) прибыли, руб.",-125_136_014.00,28_219_414.00,-171_243_342.00
"Верхняя граница дов. интервала (97.5%) прибыли, руб.",387_219_454.00,423_869_443.00,354_297_755.00
"Риск убытка, %",6.10,1.70,8.60


### Промежуточные выводы

1. Для каждого региона методом bootstrap сформированы 1000 подвыборок размером 500 скважин каждая из которых случайным образом отобрано по 200 скважин с максимальными запасами нефти спрогнозированные моделью по которым расчитаны:
   - средняя прибыль
   - прибыль на границах доверительных интервалов (2,5 и 97,5 квантили)
   - вероятность получения убытка
2. Максимальная средняя прибыль, и минимальная вероятность получить убыток оказались в регионе 2: ~441 млн. руб. и ~2% соответственно. В 1 и 2 регионах вероятность понести убытки ~>7%, средняя прибыль ~369 и ~362 млн. руб. соотвествтенно.
3. К разработке рекомендован регион 2 по причине наивысшей потенциальной доходности (441 млн. руб.) и наименьшего риска понести убытк (2%).